In [1]:
import os, osmnx as ox, geopandas as gpd, pandas as pd
ox.config(log_console=True, log_file=True, use_cache=True)
%matplotlib inline

## Configure the script

In [2]:
network_type = 'drive_service'           #get drivable public streets + service roads/alleys
retain_all = True                        #retain all components of graph, not just connected
crs_osm = {'init':'epsg:4326'}           #crs that osm uses
input_folder = 'input_data/shapefiles'   #the input urban areas shapefiles folder
input_shp_filename = 'urban_edge_t3.shp' #the input shapefile name
graphs_folder = 'output_data/graphs'     #output folder to save graphs
output_path = 'output_data/stats'        #output folder to save the stats

## Download the street networks with OSMnx

Save them to shapefiles and GraphML

In [3]:
for shapefile_folder in os.listdir(input_folder):
    if not os.path.exists('{}/{}/{}.graphml'.format(graphs_folder, shapefile_folder, shapefile_folder)):
        gdf = gpd.read_file('{}/{}/{}'.format(input_folder, shapefile_folder, input_shp_filename))
        gdf = gdf.to_crs(crs_osm)
        geometry = gdf.unary_union
        G = ox.graph_from_polygon(polygon=geometry, network_type=network_type,
                                  name=shapefile_folder, retain_all=retain_all)
        ox.save_graph_shapefile(G, folder=graphs_folder, filename='{}/shapefiles'.format(shapefile_folder))
        ox.save_graphml(G, folder=graphs_folder, filename='{}/{}.graphml'.format(shapefile_folder, shapefile_folder))

## Calculate stats

In [4]:
rows = []
for folder in os.listdir(graphs_folder):
    for item in os.listdir('{}/{}'.format(graphs_folder, folder)):
        if '.graphml' in item:
            
            # load each graphml file
            G = ox.load_graphml(filename=item, folder='{}/{}'.format(graphs_folder, folder))
            
            # load the urban area boundary shapefile and calculate total area
            gdf = gpd.read_file('{}/{}'.format(input_folder, folder))
            area_m2 = gdf.unary_union.area
            area_km2 = area_m2 / 1e6
            
            # calculate network stats with osmnx and add as new dataset row
            stats = ox.basic_stats(G, area=area_m2)
            row = {}
            row['name'] = folder
            row['area_km2'] = area_km2
            row['intersect_count'] = stats['count_intersections'] 
            row['street_length_avg'] = stats['street_length_avg']
            row['intersect_density_3way'] = stats['streets_per_node_counts'][3] / area_km2
            row['intersect_density_4way'] = stats['streets_per_node_counts'][4] / area_km2
            row['circuity_avg'] = stats['circuity_avg']
            row['streets_per_node_avg'] = stats['streets_per_node_avg']
            row['streets_per_node_stddev'] = pd.Series(G.graph['streets_per_node']).std()
            rows.append(row)

In [5]:
# convert stats dataset to dataframe and re-order columns to move name to the front
df = pd.DataFrame(rows)
cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index('name')))
df = df.reindex(columns=cols)

In [6]:
# save stats as csv
df.to_csv('{}/stats.csv'.format(output_path), encoding='utf-8', index=False)
df.head()

,name,area_km2,circuity_avg,intersect_count,intersect_density_3way,intersect_density_4way,street_length_avg,streets_per_node_avg,streets_per_node_stddev
0,Belo_Horizonte,2245.164249,1.039951,48627,15.388184,5.945667,110.990909,3.093634,0.793810
1,Bogota,2023.481175,1.038339,55991,19.268773,7.530092,78.107412,2.964267,0.898273
2,Buenos_Aires,6245.189028,1.010880,165175,11.666901,14.501082,103.933234,3.437733,0.753066
3,Cabimas,1926.709741,1.037303,7789,3.323801,0.713133,112.098145,2.817033,0.882992
4,Caracas,1487.630080,1.175846,11518,6.228699,1.402230,144.780658,2.729979,0.976056
